Sai Saradha Kalidaikurichi Lakshmanan
EECS 531- Final Project Report

#                    <center><b>Building a Visual Question Answering System</b></center>

# Abstract

A Visual Question Answering (VQA) system was built and the performance of two different neural network models - Baseline Multi Layer Perceptron (MLP)<sup>1</sup> and Long Short Term Memory (LSTM)<sup>1</sup> - were compared with respect to their accuracy and loss. VQA is an interesting and complex AI task involving multiple disciplines such as Computer Vision, Natural Language Processing and Machine Learning where a machine is trained to answer questions about an image shown to it. In this project, I have used a deep learning approach to setup the VQA system by training the neural networks on the VQA V2.0 open-ended dataset<sup>2</sup>. Training was implemented on Google Cloud VM instances and evaluation of the trained model was performed locally. Training both the models for just 50 epochs resulted in promising results with accuracy values reaching 25.34% for MLP and 22.15% for LSTM, however, preliminary observations in extended training showed indications that LSTM outperforms MLP (data not shown).

# Introduction

### Overview
In the past few years, building solutions to AI problems involving multi-modal knowledge has emerged as an important topic of interest in the machine learning community.  Visual Question Answering system (VQA) is one such area that has gained wide interest in the recent years<sup>3</sup>. This task is considered to be AI-complete and it has a wide range of potential applications, such as, helping visually challenged people for instance. VQA involves understanding several features of images in substantial detail and then be able to answer questions about the image. Therefore, the input is a question-image pair and the output is a natural language answer. The most challenging part of building a VQA system is the dataset and the open-endedness of the entire setup<sup>3</sup>. Questions and answers are free-form and open ended, thereby, making it a difficult task. 

### Related Work

There has been numerous advances in the past few years on and around building a VQA system. Currently, the most widely used methods for VQA are deep learning based and non-deep learning models, such as regression based classifiers<sup>4</sup> are still in their early stages of development. The model in reference [5] is similar to our baseline MLP model but differ in that they first extract features from pretrained convolutional neural networks (CNN) and the word embedding vectors for text data are concatenated and then regression is performed. The LSTM based models (similar to the one used in this project) were later found to perform better than CNN + Bag of Words and related techniques. In these models, LSTM is used for text data (conditioning the question) and then combined with the image features. A more recent approach is using attention based models<sup>3</sup>. Attention techniques have been quite popular in this domain and they are reported to outperform other methods. It uses two forms of attention - Parallel and Alternating attention and the primariy difference between the two is how the question and image attend over each other. In parallel attention, the question and image attend over each other simultaneously while in the alternative approach, the image is iteratively attended followed by question and image and so on. Other contributions to VQA also comes from two related tasks - Text based question answering systems and Image captioning.

### Technical Background

In this project, we have used a deep learning approach and hence a short description of three main types of neural networks used in the architecture is given below.

#### Multi Layer perceptrons
Multilayer perceptrons are a type of feedforward artificial neural networks (ANN) with linear/non-linear activation functions (Fig.1). They learn the target concept by backpropagating the gradients and updating them to reduce a loss function. There are different types of loss functions and in this project we have used cross-entropy as it is one of the commonly used loss functions in VQA problems. The non-linear activation functions such as sigmoid, tanh and relu are commonly used. In this project we have used tanh non-linearity as mentioned in reference [1]. The goal of these networks is to learn the weights and biases that optimize the objective function (loss function) and learn the target concept correctly.

![Alt text](imgs/MLP_MG.JPG?raw=true "MLP_MG")

<center><b>Figure 1:</b>Schematic of a MLP ANN</center>

#### Convolutional Neural Networks (ConvNets) 
ConvNets are primarily used for images and have 3D neurons (the third dimension being along the depth). Each ConvNet has one or more of the following layers (Fig.2) - Input, Conv, ReLu or activation layer, Pooling layer (eg. max pooling) and Fully Connected (FC) layer. A thorough description of ConvNets and the properties of the individual layers are convered in reference [6]. 

![Alt text](imgs/CNN_MG.JPG?raw=true "CNN_MG")
<center><b>Figure 2:</b>Schematic of a ConvNet</center>

<b>VGGNet</b>: The image features in our project are generated from a pretrained VGG neural net<sup>7</sup>. Therefore, we briefly describe the architecture of the VGG net here. The ImageNet challenge<sup>8</sup> triggered the development of many of the popular neural net models that are in wide use today and one of them is the 16 layer VGG net by Simonyan and Zisserman<sup>7</sup>. The architecture of this neural network is shown below (Fig.3a,b):

![Alt text](imgs/vgg_MG.JPG?raw=true "vgg_MG")
<center><b>Figure 3:</b>Schematic of a VGGNet</center>

This network has about 16 Conv/FC layers and has 3x3 convolutions in all layers. The disadvantage of this network is the large number of parameters involved (in the order of 1.6M) which made it too slow and expensive. There has been several models that were developed later for reducing the parameters and still remain deep with good accuracy (for example reference [9])

#### Long Short Term Memory (LSTM) RNNs
LSTMs are a type of recurrent neural networks (RNN) that act as a solution for the long-term dependency problem of RNNs. Specifically, in certain instances, it is insufficient to just remember the input at previous time step alone, instead the networks may have to remember context from several previous timesteps and this is a problem with vanilla RNNs. LSTMs have a chain like structure (Fig. 4) with each repeating module in the structure consisting of four type of layers and gate like structures to control the cell state. The forget gate layer is the first layer that decides what information passes on to the next state and this is a sigmoid layer. Once the RNN has decided what to forget, the next step is to decide what to store and that is taken care of by the input gate layer following which a state update happens. The output of the cell is given by passing this updated state value through a sigmoid function and multiply that with the same cell state passed through a tanh function. 

![Alt text](imgs/LSTM_2.png?raw=true "LSTM_2")
<center><b>Figure 4:</b>Schematic of the LSTM RNN[]</center>


# Environment Setup and Methods

## Dataset
We have used the VQA V2.0 dataset for training and evaluating the models<sup>2</sup>. VQA V2.0 includes about 3.8M questions on about 82,000 images and the validation set has about 2M questions on about 40,000 images. The earlier version of this dataset had abstract scene image-question-answer triplets, but V2.0 uses only the MS COCO dataset<sup>10</sup>. The COCO dataset images were annotated by  mechanical turks. The dataset questions and answers are open-ended and designed in a way that requires the image to be input for the machine to provide the answer (instead of asking any generic question). The dataset has questions of different types - What, Where, How much/many, When, etc., According to the team that did the dataset collection<sup>1,2</sup>, about 89% of the data has single word answers and most of them containing less than or utmost 3 words. About 38% of the training data includes yes/no type questions and this means that they are the majority values. They also say that about 58% of yes/no questions are yes (and this also shows in our interesting observation while evaluating the trained model discussed in the results section). The ground truth for test data is not provided hence we used the validation data for testing the models. Some of the other popular datasets are DAQUAR, Visual7W and Coco-QA. Sample images from VQA dataset are shown below (Fig.5).

![Alt text](imgs/sampleimage_MG.JPG?raw=true "sampleimage_MG")
<center><b>Figure5.</b>Sample images from the VQA V2.0 dataset</center>

For generating the image features, the pretrained VGGNet model was used. Infact, the features generated from the VGG16 net is available as .mat file (MATLAB data) for download and this saves some time during training and evaluation since we now directly have the features. For embedding the text data and generating the word vectors, we used the Stanford's Glove Model<sup>11</sup>. This model seems to have produced great results and we used the model that was trained on Common Crawl dataset with 2M vocab and 8B tokens. Loading this word vec dataset before beginning training and evaluation was a bit long but helped neverthless.

## Machine requirements
All of the training for the two models were done in Google Cloud. We used two VM instances on the Google Cloud (one with 8core CPU and the other with a NVIDIA Tesla K80 GPU). These cloud compute engines are charged per hour and we get a free $300 credit and fortunately we were able to run the project without exhausting the credits! Both the instances ran on Ubuntu 16.04 and we used Python3.6 and Tensorflow 1.7 on the instance. Evaluation is faster hence evaluation were done locally on a Windows 7 (x64) machine equipped with 8GB RAM.

## Method

Both the models used here are described in reference [1]. While, they are not the current state of art, they serve as good models for preliminary exploration into the VQA domain. The input to these networks are question-image pairs. For training, the answers are also input to compute the cross entropy loss over iterations. In both cases we begin by first loading the Glove model data (word-vectors for over 2.2 M vocab words). Then from the question-answer-image triplet, we only select those that have the top 1000 frequent answers from the training dataset. As stated in reference [1], these top 1000 frequent answers contribute to about 80% of the answers to the VQA dataset, hence we only took those records for training. We then load the VGGNet features for the COCO images and then start building the corresponding models.

### Approach 1 - MLP (BoW + CNN + 2 layer MLP) 
We followed the implementation similar to that mentioned in reference [1]. The network model has 2 hidden layers with dropout and tanh activation (Fig.6). First, each token in the question has a corresponding 300 dimensional word vector that are summed up together and form the bag of features. The image features generated from VGGNet are 4096 dimensional each and before feeding the input to the MLP model, these two set of feature vectors are concatenated (resulting in 4396 dimensional input vector). Each hidden layer had 1024 neurons and the final output layer also has a softmax estimation and the cross entropy loss is calculated from that. For prediction, argmax(softmax normalization) gives the corresponding class (1000 classes/answers as described earlier). The graph of the network designed for this study (using TensorFlow) is shown in Figure 7. Figure 8 shows the zoomed graph with the variables and operations involved in the computation.

![Alt text](imgs/model_1.PNG?raw=true "model_1")
<center><b>Figure 6.</b> MLP Model</center>

![Alt text](imgs/graph_full.PNG?raw=true "graph_full")
<center><b>Figure 7.</b> Network Graph used in this study (TensorFlow graph)</center>

![Alt text](imgs/graph_zoomed.PNG?raw=true "graph_zoomed")
<center><b>Figure 8.</b> A zoomed-in representation of the network Graph from Fig.7</center>

### Approach 2 - LSTM
In this method, we first build a single layer LSTM for embedding the questions (Fig. 9). A single layer LSTM's final state (512 dimensional) embedding for the query is then concatenated with the 4096 dimensional image feature vector and input to three fully connected layers (each with 1024 neurons) with dropout (0.5) and tanh activation. The batch size for both models was 256.

![Alt text](imgs/model_2.PNG?raw=true "model_2")
<center><b>Figure 9.</b> LSTM Model </center>

![Alt text](imgs/graph_full.PNG?raw=true "graph_full")
<center><b>Figure 10.</b> LSTM Network Graph built for this study (using TensorFlow)</center>

Once these models have been built and trained, they are evaluated on the validation dataset. The model with graph metadata and the weights were saved in the directory and then evaluated locally by loading the model in tensorflow default graph. The softmax predictions from the output layer is taken and argmax of it gives the class. Once the class is obtained, we do an inverse transform of the word embedding to obtain the actual response to the question.

# Experimental Results

The performance metric used is accuracy. To calculate accuracy, the recommended evaluation metric stated in reference [1,2] is,

<b>Accuracy(response)</b> = min((#of humans that gave the specific response)/3, 1)

This means that to consider the algorithm's response to be fully correct (implying score of 1), atleast 3 of the 10 humans should have given that answer. If less than three people have given the same answer, then (# of human answered response)/3 is chosen as the score. The final accuracy is obtained by summing up all scores and dividing by the total length of questions.

## Results
The time taken to train each epoch for batch size of 256 was about 6 minutes on VM Instance 1 with CPU only for MLP and about 8 minutes on VM Instance 2 with GPU. Since we had only one GPU, it was not extremely efficient in speeding up the training process. Both models were trained only for 50 epochs to accommodate the constraint on time and free-cloud space allocation. 

### Loss Function 
As we can observe from the graphs below, the loss has consistently decreased over epochs.

<b>a. MLP:</b>

![Alt text](imgs/loss_all.PNG?raw=true "loss_all")
<center><b>Figure 11.</b>Loss value versus Number of Iterations showing consistent decrease in average loss over iterations for MLP</center>
  
In the graph below (Fig. 12), we plot a custom graph (written using a custom TensorFlow routine) that displays the change in weight matrix, for layer 1 of the MLP network, over iterations. 

![Alt text](imgs/weight_over_iterations.PNG?raw=true "weight_over_iterations")
<center><b>Figure 12.</b>Layer 1  weight matrix statistics</center>

Figure 13 are the PCA results (default TensorFlow analysis) for the weight matrix of output layer. 

![Alt text](imgs/weights_out_PCA.PNG?raw=true "weights_out_PCA")
<center><b>Figure 13.</b>PCA results for the weight matrix of output layer</center>

Finally, Figure 14 shows the a snapshot of the final output VM instance screen at the end of the training phase displaying the loss value at each iteration.

![Alt text](imgs/Final_output.PNG?raw=true "Final_output")
<center><b>Figure 14.</b>Final output snapshot of training phase</center>


<b>b. LSTM:</b>

![Alt text](imgs/loss_over_iter.PNG?raw=true "loss_over_iter")
<center><b>Figure 15.</b>Loss value versus Number of Iterations showing consistent decrease in average loss over iterations for LSTM</center>

### Sample Outputs from the VQA evaluation

The images shown below are examples of outputs from the VQA evaluation.  The response to the image-question pair input to the trained model show the efficiency of the model in answering open-ended questions. Note that some of the answers are more than 1 word. 

![Alt text](imgs/out1.JPG?raw=true "out1")
![Alt text](imgs/out2.JPG?raw=true "out2")
![Alt text](imgs/out3.JPG?raw=true "out3")
![Alt text](imgs/out4.JPG?raw=true "out4")

The images shown below are examples of incorrect responses from the model. 

![Alt text](imgs/Figure_1-5.png?raw=true "Figure_1-5")

![Alt text](imgs/Figure_1-6.png?raw=true "Figure_1-6")


# Observation

One interesting observation from the results is that the model seemed to have learnt the 'yes' response quite well. Out of the correct responses, 85% of them were answered as 'yes' and only the remaining 15% of the right responses were general language answers and also numbers. This is only from training the model for 50 epochs and the model's steady learning process shows that given more epochs of training, it would definitely perform better and equal to the state of art accuracy measures. Similarly LSTM seems to have learnt the 'no' response, as more than 85% of the right responses were answered as no. But these results are only with 50 epochs. We were able to run the training for 100 epochs and saw good drops in the cross entropy loss, however due to an internet disconnect problem the model did not get saved and we were unable to run the evaluation for the training done on 100 epochs. We would also like to do a lot of variations to the models such as changing the non-linearities, varying dropout raitos, etc., to see how the performance improves, but we did not have enough time to run those experiments and include the results in this report. But we will continue to experiment with the models. While the current state of art in VQA is about 50%, the accuracy of our model (~ 25%) is expected given that the model was trained only for 50 epochs.

# Summary

In this project, we explored the VQA problem by implementing two models (MLP and LSTM) and obtained appreciable accuracy with only 50 epochs. We found that in the training phase (~50 epochs) the MLP model favored the yes response. We would like to continue to work on these models and improving them by finding better values for the parmaters of the network. As future work, we would also like to try the more recent MCB model<sup>12</sup> and Attention based models<sup>13</sup> and compare their performances.

### References

1. Agarwal, A., et al. "VQA: Visual Question Answering", arXiv:1505.00468 [cs.CL], (2016)
2. visualvqa.org
3.  Gupta, A.K., "Survey of Visual Question Answering: Datasets and Techniques",arXiv:1705.03865 [cs.CL], (2017)
4.  K. Kafle and C. Kanan, "Answer-type prediction for visual question answering", The IEEE Conference CVPR, (2016)
5.  Zhou, B., et al. "Simple Basline for Visual Question Answering", arXiv:1512.02167v2 [cs.CV], (2015)
6. http://cs231n.github.io/convolutional-networks/
7.  Simonyan, K and Zisserman, A. "Very Deep Convolutional Networks for Large-Scale Image Recognition",arXiv:1409.1556 [cs.CV] (2015)
8. Russakovsky, O., et al. "ImageNet Large Scale Visual Recognition Challenge",arXiv:1409.0575 [cs.CV] (2015)
9. Lin, M., et al. "Network in Network", arXiv:1312.4400 [cs.NE] (2014)
10. http://cocodataset.org/#home
11.  https://nlp.stanford.edu/projects/glove/
12. Fukui, A., et al."Multimodal Compact Bilinear Pooling for Visual Question Answering and Visual Grounding",arXiv:1606.01847 [cs.CV]
 (2016)
13. Lu, J., et al. "Heirarchical Question-Image Co-Attention for Visual Question Answering", arXiv:1606.00061 [cs.CV] (2017)